# Team Prison Escape
You and your team are trying to escape from an enemy prison. The prison is secured by a combo lock that you must work together to break. The lock requires each team member to guess and input a secret digit from 1-9 simultaneously.

## Rules
1. Your team has up to 15 attempts at the lock. On each attempt, every team member guesses a digit from 1-9.
2. After all the guesses are entered, the lock uses the guesses to compute a "team total". It will use the secret digits to compute a "magic total". If the team total is close enough to the magic total (within 5), then the attempt is successful. 3 successful attempts in a row will break the lock.
3. However, the lock is **intelligent** and changes the magic total and correct total after every attempt! Despite that, the secret digits that each player is trying to guess will stay the same.
4. [Hard mode] No discussion allowed - you can't alert the prison guards...

## Clues
Beating an intelligent lock sounds impossible, right? Fortunately, one of your team members (your TA) is an AI expert and has figured out some clues about how the lock works:

1. After the guesses are entered, the lock multiplies each player's guess by a unique *multiplier*. It then computes the team total by adding the multiplied guesses together.
2. The same multipliers are used to compute the magic total, except by using the secret digits instead of the guesses. The lock also appears to add some random noise (usually a small number between -3 and 3) to the magic total.
3. The multipliers and noise value are the only things that change every round.

Your TA was able to hack the lock, and get it to display the multipliers in addition to the secret total after each attempt. Every team member can use this information to update their guesses for the next attempt.

**Ready to try? Run the cell below to get started.**

In [49]:
import ipywidgets as widgets
from ipywidgets import interact, Layout
from IPython.display import display 
import random

## Setup: Constants
MIN_PLAYERS = 1
MAX_PLAYERS = 12
MIN_DIGIT = 1
MAX_DIGIT = 9
MIN_MULTIPLIER = -2
MAX_MULTIPLIER = 5
SIGMA = 1
MAX_ATTEMPTS = 15

## Setup: Global Variables
global n
global player_names
global secret_digits

global attempt_number
global guesses
global multipliers
global noise_term

## Setup: Helper Functions
def generate_secret_digits():
  global n, secret_digits, MIN_DIGIT, MAX_DIGIT
  secret_digits = [
    random.randint(MIN_DIGIT, MAX_DIGIT)
    for i in range(n)
  ]

def attempt_header_text():
  global attempt_number
  return f"Lock Attempt #{attempt_number}"

def cell_text(value):
  return f"<span>{str(value)}</span>"

def generate_multipliers():
  global n, multipliers, MIN_MULTIPLIER, MAX_MULTIPLIER
  multipliers = [
    random.randint(MIN_MULTIPLIER, MAX_MULTIPLIER)
    for i in range(n)
  ]

def generate_noise_term():
  global SIGMA, noise_term
  noise_term = min(3, int(random.normalvariate(0, SIGMA)))
  noise_term = max(-3, noise_term)

## Setup: Layouts
num_players_input = widgets.BoundedIntText(value=3, min=MIN_PLAYERS, max=MAX_PLAYERS)
num_players_submit = widgets.Button(description="Submit", tooltip="Submit number of players")
num_players_section = widgets.VBox([
  widgets.HTML('<h2>Number of players:</h2>'),
  num_players_input,
  num_players_submit,
])

global player_names_input
player_names_submit = widgets.Button(description="Submit", tooltip="Submit player names")
global player_names_section

global lock_attempt_title
global guess_table
header_texts = ["Guess", "Multiplier", "Subtotal"]
guess_table_columns = [
  widgets.HTML(
    f"<strong>{header_texts[i]}</strong>",
    layout=widgets.Layout(width="auto", grid_area=f"col{i+1}")
  )
  for i in range(3)
]
global guess_table_rows
global guess_cells
global multiplier_cells
global subtotal_cells
separator_row = widgets.HTML('<hr></hr>')
team_total_row = widgets.HBox([
  widgets.HTML('<strong>Team Total:</strong>'),
  widgets.HTML('<strong>?</strong>'),
])
magic_total_row = widgets.HBox([
  widgets.HTML('<strong>Magic Total:</strong>'),
  widgets.HTML('<strong>?</strong>'),
])
lock_attempt_submit = widgets.Button(description="Submit Guesses", tooltip="Submit guesses")
global lock_attempt_section

## Core game event handlers
def confirm_num_players(button):
  global num_players_input, n
  n = num_players_input.value

  # Finish setup requiring n
  generate_secret_digits()
  display_player_names()

def display_player_names():
  global player_names_input, player_names_section, player_names_submit

  player_names_input = [
    widgets.Text(description=f"Name {i+1}")
    for i in range(n)
  ]
  
  player_names_widgets = [
    widgets.HTML('<h2>Player names:</h2>'),
    player_names_submit,
  ]
  player_names_widgets[1:1] = player_names_input
  player_names_section = widgets.VBox(player_names_widgets)

  display(player_names_section)

def confirm_player_names(button):
  global player_names

  player_names = [p.value for p in player_names_input]
  display_lock_attempt_section()

def display_lock_attempt_section():
  global n
  global lock_attempt_title
  global guess_table
  global separator_row
  global team_total_row
  global magic_total_row
  global lock_attempt_submit
  global lock_attempt_section

  lock_attempt_title = widgets.HTML(f"<h2>{attempt_header_text()}</h2>")
  init_guess_table()

  lock_attempt_section = widgets.VBox([
    lock_attempt_title,
    guess_table,
    separator_row,
    team_total_row,
    magic_total_row,
    lock_attempt_submit,
  ])

  display(lock_attempt_section)

def init_guess_table():
  global n, player_names, MIN_DIGIT, MAX_DIGIT
  global guess_table
  global guess_table_columns
  global guess_table_rows
  global guess_cells
  global multiplier_cells
  global subtotal_cells

  guess_table_rows = [
    widgets.HTML(
      f"<strong>{player_names[i]}</strong>",
      layout=widgets.Layout(width="auto", grid_area=f"row{i+1}")
    )
    for i in range(n)
  ]

  guess_cells = [
    widgets.BoundedIntText(
      value=1,
      min=MIN_DIGIT,
      max=MAX_DIGIT,
      layout=widgets.Layout(width="auto", grid_area=f"cell{i+1}1")
    )
    for i in range(n)
  ]

  multiplier_cells = [
    widgets.HTML(
      cell_text("?"),
      layout=widgets.Layout(width="auto", grid_area=f"cell{i+1}2")
    )
    for i in range(n)
  ]

  subtotal_cells = [
    widgets.HTML(
      cell_text("?"),
      layout=widgets.Layout(width="auto", grid_area=f"cell{i+1}3")
    )
    for i in range(n)
  ]

  all_cells = []
  all_cells.extend(guess_table_columns)
  all_cells.extend(guess_table_rows)
  all_cells.extend(guess_cells)
  all_cells.extend(subtotal_cells)
  all_cells.extend(multiplier_cells)

  grid_template_areas = ['". col1 col2 col3"']
  for i in range(n):
    grid_template_areas.append(f'"row{i+1} cell{i+1}1 cell{i+1}2 cell{i+1}3"')
  
  grid_template_areas = "\n" + "\n".join(grid_template_areas) + "\n"

  guess_table = widgets.GridBox(
    children=all_cells,
    layout=widgets.Layout(
      width="50%",
      grid_template_rows=" ".join(["auto" for i in range(n+1)]),
      grid_template_columns="25% 25% 25% 25%",
      grid_template_areas=grid_template_areas,
      grid_gap='5px 10px'
    )
  )

def confirm_lock_attempt(button):
  global attempt_number, guesses, multipliers, noise_term
  global guess_cells
  global multiplier_cells
  global subtotal_cells
  global team_total_row
  global magic_total_row

  lock_attempt_title.value = f"<h2>{attempt_header_text()}</h2>"
  attempt_number += 1

  guesses = [
    guess_cell.value
    for guess_cell in guess_cells
  ]

  generate_multipliers()
  generate_noise_term()

  subtotals = [
    guesses[i] * multipliers[i]
    for i in range(n)
  ]
  team_total = sum(subtotals)

  magic_total = noise_term + sum([
    secret_digits[i] * multipliers[i]
    for i in range(n)
  ])

  for i in range(n):
    multiplier_cells[i].value = cell_text(multipliers[i])
    subtotal_cells[i].value = cell_text(subtotals[i])

  team_total_row.children[1].value = cell_text(team_total)
  magic_total_row.children[1].value = cell_text(magic_total)

## Initialize game loop
attempt_number = 1
num_players_submit.on_click(confirm_num_players)
player_names_submit.on_click(confirm_player_names)
lock_attempt_submit.on_click(confirm_lock_attempt)
display(num_players_section)

In [50]:
print(secret_digits)
print(guesses)

[7, 1, 3]
[7, 1, 3]


## Terminology
$w_i$ - 

## Psuedocode

1. User input: how many players? --> $n$
2. User input: each player's name
3. User input: Gaussian noise variance parameter --> $\sigma$
4. Generate $n$ ground truth weights $w_1$ -> $w_n$ from 1 to 9
5. User input: estimate $\hat{w_i}$ for each user $i \in 1 \ldots n$

In a loop, until TODO: convergence criterion
6. Generate $n$ numbers $x_1$ -> $x_n$ from -2 to 5
7. Compute estimate $\hat{y} = \sum_{i=1}^{n} \hat{w_i}*x_i$ 
8. Compute actual $y = \sum_{i=1}^{n} w_i*x_i$
9. User input: re-estimate $\hat{w_i}$ for each user $i \in 1 \ldots n$

## Directions
1. Each player updates their weights independently
2. Keep iterating until one of the following criteria is met - A) at least three rounds in a row where the error is <= 3 or B) ten iterations have been reached

## Questions
1. How did you adjust your guess when the 

